In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [29]:
ratings = pd.read_json('./data/ratings.json')
ratings

,John Carson,Michelle Peterson,William Reynolds,Jillian Hobart,Melissa Jones,Alex Roberts,Michael Henry
Inception,2.5,3.0,2.5,NaN,3,3.0,NaN
Pulp Fiction,3.5,3.5,3.0,3.5,4,4.0,4.5
Anger Management,3.0,1.5,NaN,3.0,2,NaN,NaN
Fracture,3.5,5.0,3.5,4.0,3,5.0,4.0
Serendipity,2.5,3.5,NaN,2.5,2,3.5,1.0
Jerry Maguire,3.0,3.0,4.0,4.5,3,3.0,NaN


In [30]:
# 计算用户与用户的相似度矩阵
simmat = ratings.corr()  # 相关性系数[-1, 1]
simmat  # 相似性矩阵

,John Carson,Michelle Peterson,William Reynolds,Jillian Hobart,Melissa Jones,Alex Roberts,Michael Henry
John Carson,1.000000,0.396059,0.404520,0.566947,0.594089,0.747018,0.991241
Michelle Peterson,0.396059,1.000000,0.204598,0.314970,0.411765,0.963796,0.381246
William Reynolds,0.404520,0.204598,1.000000,1.000000,-0.258199,0.134840,-1.000000
Jillian Hobart,0.566947,0.314970,1.000000,1.000000,0.566947,0.028571,0.893405
Melissa Jones,0.594089,0.411765,-0.258199,0.566947,1.000000,0.211289,0.924473
Alex Roberts,0.747018,0.963796,0.134840,0.028571,0.211289,1.000000,0.662849
Michael Henry,0.991241,0.381246,-1.000000,0.893405,0.924473,0.662849,1.000000


**基于simmat相似度矩阵，使用UBCF算法实现简单推荐引擎**
+ 用户协同过滤

In [62]:
# 获取登录用户
login_user = 'Michael Henry'
# 寻找登录用户的相似用户，去掉自己，去掉反相关的用户
sim_users = simmat[login_user]  # 行列都一致数据
sim_users.drop(login_user, inplace=True)
sim_users = sim_users[sim_users>0]
# print(sim_users)

# 遍历所有相似用户，看相似用户都看过的电影
# 找个合适的数据结构，存储推荐列表，还有所有人对每一部电影的打分
movie_list = {}
for sim_user, sim_score in sim_users.items():
    movies = ratings[sim_user]
    # print(sim_user)
    # 检索一遍，哪些是登录用户没看过的，把这些电影存起来
    for movie, score in movies.dropna().items():
        if np.isnan(ratings[login_user][movie]):
            # 没看过这个电影
            if movie not in movie_list.keys():
                movie_list[movie] = [[],[]]
            movie_list[movie][0].append(score)
            movie_list[movie][1].append(sim_score)

movie_list
# 对电影列表排序
ml = sorted(movie_list.items(), key=lambda x:np.average(x[1][0],weights=x[1][1]), reverse=True)

ml[:]

[('Jerry Maguire',
  [[3.0, 3.0, 4.5, 3, 3.0],
   [0.9912407071619305,
    0.3812464258315117,
    0.8934051474415642,
    0.924473451641905,
    0.6628489803598702]]),
 ('Inception',
  [[2.5, 3.0, 3, 3.0],
   [0.9912407071619305,
    0.3812464258315117,
    0.924473451641905,
    0.6628489803598702]]),
 ('Anger Management',
  [[3.0, 1.5, 3.0, 2],
   [0.9912407071619305,
    0.3812464258315117,
    0.8934051474415642,
    0.924473451641905]])]